In [1]:
%matplotlib inline
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import pandas as pd
from datetime import datetime, date
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly as py
from plotly.offline import iplot, plot, init_notebook_mode, download_plotlyjs
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.offline as offline
import operator
%matplotlib inline
import scipy as sp
plt.style.use('fivethirtyeight') # For plots


# Dish similarity

In [2]:
md = pd.read_csv('rating.csv')


In [3]:
md = md.fillna('0')

,meal_id,meal_title,meal_formatted_title,meal_container,dish_type,meal_hot,meal_ingredient_categories,meal_proteins,meal_grams,meal_fats,meal_carbs,meal_caloricity,есть рыба,есть мясо/птица,глютен,молочка,суп,ci_lower_bound
0,448,СЫРНИКИ ТВОРОЖНЫЕ СО СМЕТАНОЙ 3ШТ/190ГР (JF),Сырники со сметаной,36,3.0,1,"глютен,молочка,сметана,яйца",15.90,190,13.02,9.73,220,0,0,1,1,0,0
1,554,СЫРНИКИ ТВОРОЖНЫЕ СО СМЕТАНОЙ 2ШТ/140ГР (JF),Сырники со сметаной,36,3.0,1,"глютен,молочка,сметана,яйца",10.95,140,10.68,6.97,168,0,0,1,1,0,0.171186
2,1,АБРИКОСЫ-СРЕДНИЙ ВЕС-120 3ШТ/103ГР (JF),Абрикосы,суповая,0,0,0,0.93,103,0.10,9.27,42,0,0,0,0,0,0
3,10032,ОМЛЕТ С БРОККОЛИ И СТРУЧКОВОЙ ФАСОЛЬЮ 1ШТ/230Г...,Омлет с брокколи и стручковой фасолью,36,3.0,1,"зелень,бобовые,глютен,злаки,капуста,молочка,пе...",14.97,230,16.15,7.36,235,0,0,1,1,0,0.637808
4,10084,ТВОРОГ С МАЛИНОЙ И МЮСЛИ 1ШТ/165ГР (JF),Творог с малиной и мюсли,десертница,6.0,0,"орехи,глютен,злаки,кукуруза,кунжут,молочка,сме...",15.53,165,19.83,24.59,338,0,0,1,1,0,0.889772


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
md['meal_ingredient_categories'] = md['meal_ingredient_categories'].fillna('')
tfidf_matrix = tfidf.fit_transform(md['meal_ingredient_categories'])
tfidf_matrix.shape

(1826, 33)

In [6]:
# the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# a reverse map of indices and titles
indices = pd.Series(md.index, index=md['meal_ingredient_categories']).drop_duplicates()
def get_recommendations(meal_formatted_title, cosine_sim=cosine_sim):
    idx = indices[meal_formatted_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return md['meal_ingredient_categories'].iloc[movie_indices]

In [7]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(md['meal_formatted_title'])
tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]
md = md.reset_index()
titles = md['meal_formatted_title']
indices = pd.Series(md.index, index=md['meal_formatted_title'])
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    meals_indices = [i[0] for i in sim_scores]
    return titles.iloc[meals_indices]
get_recommendations('Творог с малиной и мюсли').head(10)

1063                   Мюсли с вишней
1350                  Творог с вишней
1351                  Творог с вишней
1619           Рисовая каша с малиной
1384    Запеканка творожная с малиной
1389    Запеканка творожная с малиной
856           Конфета-мюсли с бананом
857           Конфета-мюсли с бананом
1315          Конфета-мюсли с бананом
1305        Смузи с бананом и малиной
Name: meal_formatted_title, dtype: object

In [8]:
md['dish_type'] = md['dish_type'].astype('int')
md['ci_lower_bound'] = md['ci_lower_bound'].astype('int')
s = md.apply(lambda x: pd.Series(x['meal_ingredient_categories']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'meal_ingredient_categories'
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0)
tfidf_matrix1 = tf1.fit_transform(md['meal_ingredient_categories'])
tfidf_matrix1.shape
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
cosine_sim1[0]
md1 = md.reset_index()
titles1 = md1['meal_title']
indices1 = pd.Series(md.index, index=md['meal_title'])
def get_recommendations(title):
    idx = indices1[title]
    sim_scores1 = list(enumerate(cosine_sim1[idx]))
    sim_scores1 = sorted(sim_scores1, key=lambda x: x[1], reverse=True)
    sim_scores1 = sim_scores1[1:]
    meals_indices1 = [i[0] for i in sim_scores1]
    return titles1.iloc[meals_indices1]
get_recommendations('ТВОРОГ С МАЛИНОЙ И МЮСЛИ 1ШТ/165ГР (JF)').head(15)

1063                        МЮСЛИ С ВИШНЕЙ 1ШТ/180ГР (JF)
168                     ЗАПЕКАНКА С ВИШНЕЙ 1ШТ/145ГР (JF)
1372    КАША ПШЕННАЯ С КИНОА, РИСОМ И БАНАНОВО-ВИШНЕВЫ...
36              ДЕСЕРТ ТВОРОЖНЫЙ СО СЛИВОЙ 1ШТ/190ГР (JF)
57                 ТВОРОГ С ЯГОДНЫМ МИКСОМ 1ШТ/190ГР (JF)
58                 ТВОРОГ С ЯГОДНЫМ МИКСОМ 1ШТ/250ГР (JF)
1585              МУСС ТВОРОЖНЫЙ С ЯГОДАМИ 1ШТ/215ГР (JF)
25          ЗАПЕКАНКА ТВОРОЖНАЯ С ЧЕРНИКОЙ 1ШТ/130ГР (JF)
1554      КЛАССИЧЕСКАЯ ТВОРОЖНАЯ ЗАПЕКАНКА 1ШТ/130ГР (JF)
1451                         ТАРТ С ТЫКВОЙ 1ШТ/150ГР (JF)
1452                         ТАРТ С ТЫКВОЙ 1ШТ/100ГР (JF)
239        ТВОРОЖНАЯ ЗАПЕКАНКА С ГОЛУБИКОЙ 1ШТ/177ГР (JF)
1764             ПУДИНГ ЧИА С МАНГО И КИВИ 1ШТ/215ГР (JF)
1765                ПУДИНГ ЧИА С ШОКОЛАДОМ 1ШТ/145ГР (JF)
1786    ЛЕНИВАЯ ОВСЯНКА С БАНАНОМ И СЕМЕНАМИ ЛЬНА 1ШТ/...
Name: meal_title, dtype: object